In [410]:
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader
import shutil
from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.schema import Document
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from PyPDF2 import PdfReader
from langchain_community.document_loaders import DirectoryLoader
from langchain.chains import RetrievalQA, LLMChain

In [411]:
import os
api_key = os.getenv("OPENAI_API_KEY")

os.environ["OPENAI_API_KEY"] = api_key
#print(api_key)
FAISS_PATH = "/faiss"

In [412]:
def get_document_loader():
    loader = DirectoryLoader('documents', glob="**/*.pdf", show_progress=True, loader_cls=PyPDFLoader)
    docs = loader.load()
    return docs

docs = get_document_loader()

100%|██████████| 2/2 [00:06<00:00,  3.35s/it]


In [413]:
def get_text_chunks(documents: list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
        
    )
    chunks = text_splitter.split_documents(documents)
    return chunks

chunks = get_text_chunks(docs)

In [414]:
documents = get_document_loader()
chunks = get_text_chunks(documents)
db = FAISS.from_documents(
    chunks, OpenAIEmbeddings()
)
db

100%|██████████| 2/2 [00:05<00:00,  2.91s/it]


In [415]:
retriever = db.as_retriever()

In [416]:
llm = ChatOpenAI(model="gpt-3.5-turbo")

In [417]:
context = db.similarity_search("Give me list of documents required for claim approval")
print(context)

[Document(page_content='Obtain medication  from: Clinic  Pharmacy  \nRequire d claim documents:  Original  \nmedicatio n receipt  Prescri ption Original  \nmedication receipt  \nDocuments issued by:  Clinic  Clinic  Pharmacy  \nIssue d ate \uf0fc  \uf0fc  \uf0fc  \nPatient’s name  \uf0fc  \uf0fc  \uf0fc  \nService provider’s name and address  \uf0fc  \uf0fc  \uf0fc  \nDiagnosis  \uf0fc  \uf0fc   \nDescrip tion of med icines  \uf0fc  \uf0fc  \uf0fc  \nCharg es nature and breakd own \uf0fc   \uf0fc  \nOfficial stamp  and/or  signature of \nservice provider  \uf0fc  \uf0fc  \uf0fc', metadata={'source': 'documents\\MembershipHandbook.pdf', 'page': 19}), Document(page_content='Claim procedures  \nClaimin g Clinical Benefit,  Maternity Benefit and D ental Benefit  \n(Not using BHN Card)  \n \n 17  Other  useful tips: \n\uf09f To claim for reimbursement of the expenses for consultations with an Obstetrician and prenatal \nor postna tal check -ups on account of p regna ncy under Maternit y Ben

In [418]:
context2 = db.similarity_search(f"give me list of all general exclusions")
print(context2)

[Document(page_content='General exclusions  \n \n \n 20  14. Senile Dementia (inclu ding Alzheimer’s disease) and Parkinson’s dis ease . \n15. Any charges for the procurem ent or use of  spec ial braces a nd appliances , including but not \nlimited to spe ctacle s, hearing  aids and other equipments such as wheel chairs and crutches.  \n16. Any treatment or inves tigation related to dental or gum conditions unless it is covered under \nDental Benefit or Em ergen cy treatment  arising from Accidents or the extra ction of impacted  \nwisdom te eth during  Hospital Confinement. Fo llow-up treatment from such Hospital \nConfinement shall not be covered unless it is payable und er Dental Benefit . \n17. Treatment arisi ng from war, invas ion, acts of  foreign enemies, hostilities (wheth er war be \ndeclare d or not),  civil war , rebellion, revolution,  insurrection or military or usurped power or \nterrorist acts. \n18. Non-medical services , including b ut not limited to guest meals, rad 

In [419]:
PROMPT = """your tasked to create a machine learning model to verify if the health insurance claim should be accepted or rejected. 
the end output is to automate the process of verification of documents.

below is the documents and information required to accept the medical insurance claim at bupa insurance company. your job is to check if the patient has provided all necessary information and documents. 
read the necessary details from the booklet: {context}.
verify if the patient has provided all necessary information and all necessary documents
and if you find any incomplete information or required documents are not provided then set INFORMATION criteria as FALSE. 
if patient has provided all required documents then set INFORMATION criteria as TRUE. 
write list of all required documents for accepting the claim and write if the patient has provided the required documents


below is the list of the general exclusions of the diseases or conditions that are excluded from the scheme:{context2}.
If any disease of the patient is in the above list of general exclusions set EXCLUSION criteria as FALSE.
If any disease of the patient is not in the above list of general exclusions set EXCLUSION criteria as TRUE.


Following is the basic information of the patient: {information}. 
Use this information to verify if the application is valid and to accept or reject the application. 

if INFORMATION is FALSE reject the claim. if EXCLUSION is FALSE reject the claim. if INFORMATION is TRUE and EXCLUSION is also TRUE then only accept the claim.
Generate a detailed report about
the claim and procedures you followed for accepting or rejecting the claim and the write the information you used for creating the report. 
Create a report in the following format
-----
write weather INFORMATION AND EXCLUSION are true or false 
reject the claim if any of them is false.
write weather claim is accepted or not

Executive Summary
[In Executive Summary provide a Summary of the report.]

Introduction
[In Introduction write a paragraph about the aim of this report, what is the current situation of the claim approval.]

Claim Details
[In Claim Details provide details about the submitted claim]

Claim Description
[Write a short description about claim]

Document Verification
[This section mentions which documents are Submitted and if they are verified.] 

docuement summary 
[In this section write everuthing inside the document, what are the medical reports of the patient]

Please verify for any signs of fraud in the submitted claim if you find the documents required for accepting the claim for the medical treatment.
"""

prompt = PromptTemplate(input_variables=["context", "context2", "information"], template=PROMPT)

print(prompt)

input_variables=['context', 'context2', 'information'] template='your tasked to create a machine learning model to verify if the health insurance claim should be accepted or rejected. \nthe end output is to automate the process of verification of documents.\n\nbelow is the documents and information required to accept the medical insurance claim at bupa insurance company. your job is to check if the patient has provided all necessary information and documents. \nread the necessary details from the booklet: {context}.\nverify if the patient has provided all necessary information and all necessary documents\nand if you find any incomplete information or required documents are not provided then set INFORMATION criteria as FALSE. \nif patient has provided all required documents then set INFORMATION criteria as TRUE. \nwrite list of all required documents for accepting the claim and write if the patient has provided the required documents\n\n\nbelow is the list of the general exclusions of t

In [420]:

information = ["""Patient Name:	Mr. Satyajit Pattnaik
Patient Address:	Tung Chung, Hong Kong
Claim Details 
Claim Number: CAC-2023-13245.
Claim Type: Health Insurance
Claim Reason: Parkinson's Disease
Date of Service: March 18, 2024
Medical Facility: XYZ Hospital
Medical Provider: Dr. John Smith
Total Claim Amount: $5,000
Description: Patient with diagnosed Parkinson's underwent an stereotactic surgery and is seeking reimbursement for medical expenses totaling $5000.

Medical Bill - 
Patient Name: Mr. Satyajit Pattnaik
Patient Address: Tung Chung, Hong Kong

Description	Amount
Consultation Fee	$200
Diagnostic Tests (MRI, CT scan, etc.)	$800
Stereotactic Surgery	$3,500
Medications	$300
Hospital Room Charges	$1,200
Miscellaneous Expenses - (e.g., supplies)	$0
Total Claim Amount	$5,000

Medical Report -

Patient Information:

Name: Mr. Satyajit Pattnaik
Date of Birth: January 15, 1965
Gender: Male
Address: Tung Chung, Hong Kong
Date of Service: March 18, 2024
Medical Facility: XYZ Hospital
Medical Provider: Dr. John Smith

Chief Complaint:
Mr. Pattnaik presents with a diagnosis of Parkinson's disease and is scheduled for stereotactic surgery due to worsening symptoms and inadequate response to conservative management.

History of Present Illness:
Mr. Pattnaik is a 59-year-old male with a past medical history significant for Parkinson's disease diagnosed 8 years ago. He has been experiencing progressive tremors, bradykinesia, rigidity, and postural instability despite optimal medical management, including levodopa therapy. These symptoms have significantly impaired his quality of life and daily functioning.

Past Medical History:

Parkinson's disease (diagnosed in 2016)
Hypertension
Hyperlipidemia

Medications:

Levodopa/carbidopa 250/25 mg, 3 times daily
Lisinopril 10 mg daily
Atorvastatin 20 mg daily
Allergies:
No known allergies

Physical Examination:
General: Alert and oriented x3, cooperative. No acute distress.
Neurological: Presence of bilateral resting tremors, bradykinesia, cogwheel rigidity, and postural instability. Hoehn and Yahr stage 3.
Other Systems: Within normal limits

Diagnostic Workup:
MRI Brain: Mild generalized cerebral atrophy, otherwise unremarkable.
CT Scan: No acute intracranial abnormalities. Mild chronic small vessel ischemic changes.
Other tests: Routine blood work within normal limits.

Treatment Plan:
Mr. Pattnaik underwent stereotactic surgery on March 18, 2024, under the care of Dr. John Smith. The procedure involved bilateral deep brain stimulation targeting the subthalamic nucleus. The surgery was performed without complications.

Postoperative Course:
Mr. Pattnaik was monitored closely postoperatively for any signs of complications. He was started on a postoperative regimen including adjustments to his anti-Parkinsonian medications. He tolerated the procedure well and showed improvement in his Parkinsonian symptoms during the hospital stay.

Follow-up Plan:
Mr. Pattnaik will follow up with Dr. John Smith in the outpatient clinic in 2 weeks for postoperative assessment and programming of the deep brain stimulation device.

Prognosis:
The prognosis for Mr. Pattnaik following stereotactic surgery is expected to be favorable in terms of symptom control and improvement in quality of life. However, continued monitoring and management of Parkinson's disease will be essential for long-term care.
Supporting Documents	 
Medical Bill	Submitted above
Medical Reports	Submitted above
Prescriptions	Not Submitted
Proof of Payment	Not Submitted
Translation	Not Submitted
Claim Submission	 
Claim Form	Not Submitted 
Additional Information	Not Submitted
"""]
print(information)

["Patient Name:\tMr. Satyajit Pattnaik\nPatient Address:\tTung Chung, Hong Kong\nClaim Details \nClaim Number: CAC-2023-13245.\nClaim Type: Health Insurance\nClaim Reason: Parkinson's Disease\nDate of Service: March 18, 2024\nMedical Facility: XYZ Hospital\nMedical Provider: Dr. John Smith\nTotal Claim Amount: $5,000\nDescription: Patient with diagnosed Parkinson's underwent an stereotactic surgery and is seeking reimbursement for medical expenses totaling $5000.\n\nMedical Bill - \nPatient Name: Mr. Satyajit Pattnaik\nPatient Address: Tung Chung, Hong Kong\n\nDescription\tAmount\nConsultation Fee\t$200\nDiagnostic Tests (MRI, CT scan, etc.)\t$800\nStereotactic Surgery\t$3,500\nMedications\t$300\nHospital Room Charges\t$1,200\nMiscellaneous Expenses - (e.g., supplies)\t$0\nTotal Claim Amount\t$5,000\n\nMedical Report -\n\nPatient Information:\n\nName: Mr. Satyajit Pattnaik\nDate of Birth: January 15, 1965\nGender: Male\nAddress: Tung Chung, Hong Kong\nDate of Service: March 18, 2024\nM

In [421]:
llmchain = LLMChain(llm=llm, prompt= prompt)
llmchain

LLMChain(prompt=PromptTemplate(input_variables=['context', 'context2', 'information'], template='your tasked to create a machine learning model to verify if the health insurance claim should be accepted or rejected. \nthe end output is to automate the process of verification of documents.\n\nbelow is the documents and information required to accept the medical insurance claim at bupa insurance company. your job is to check if the patient has provided all necessary information and documents. \nread the necessary details from the booklet: {context}.\nverify if the patient has provided all necessary information and all necessary documents\nand if you find any incomplete information or required documents are not provided then set INFORMATION criteria as FALSE. \nif patient has provided all required documents then set INFORMATION criteria as TRUE. \nwrite list of all required documents for accepting the claim and write if the patient has provided the required documents\n\n\nbelow is the lis

In [426]:
print(llmchain.run({"context": context, "context2": context2, "information": information}))

-----
INFORMATION: TRUE
EXCLUSION: TRUE
CLAIM: ACCEPTED

Executive Summary
This report verifies the validity of a health insurance claim submitted by Mr. Satyajit Pattnaik for Parkinson's disease treatment. The claim has been accepted based on the verification of necessary information and the absence of any excluded conditions.

Introduction
The aim of this report is to ensure the accuracy and completeness of the submitted health insurance claim for Mr. Satyajit Pattnaik. The current situation involves reviewing the claim details, verifying the provided documents, and checking for any excluded conditions.

Claim Details
Patient Name: Mr. Satyajit Pattnaik
Claim Number: CAC-2023-13245
Claim Type: Health Insurance
Claim Reason: Parkinson's Disease
Date of Service: March 18, 2024
Total Claim Amount: $5,000

Claim Description
Mr. Pattnaik underwent stereotactic surgery for Parkinson's disease and is seeking reimbursement for medical expenses totaling $5,000.

Document Verification
- Medica

In [423]:
"""Supporting Documents	 
Itemized Medical Bill	Not Submitted
Medical Reports	Not Submitted
Prescriptions	Not Submitted
Proof of Payment	Not Submitted
Translation	Not Submitted
Claim Submission	 
Claim Form	Not Submitted 
Additional Information	Not Submitted"""

'Supporting Documents\t \nItemized Medical Bill\tNot Submitted\nMedical Reports\tNot Submitted\nPrescriptions\tNot Submitted\nProof of Payment\tNot Submitted\nTranslation\tNot Submitted\nClaim Submission\t \nClaim Form\tNot Submitted \nAdditional Information\tNot Submitted'